# 19. Auditoria: Detecção de Anomalias em Texto (Forensic)

Em investigações forenses, procuramos e-mails ou mensagens com tom de pressão, conluio ou desvio de conduta. Podemos usar analise de sentimento e intenção.

**Objetivo:** Analisar e-mails e flagrar possíveis fraudes.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

!pip install -qU langchain langchain-openai langchain-community

In [ ]:
import os
try:
    from google.colab import userdata
except ImportError:
    userdata = None
import getpass

try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Digite sua OpenAI API Key: ")

## 1. Dataset de E-mails

Mistura de e-mails normais e suspeitos.

In [ ]:
emails = [
    "Prezados, favor enviar o relatório mensal de vendas até sexta-feira.",
    "Olha, precisamos pagar esse fornecedor HOJE. O diretor mandou. Paga logo e depois a gente vê a nota fiscal. Se não pagar, vai sobrar pra você.",
    "Segue anexo o comprovante de pagamento da taxa de renovação.",
    "Vamos fechar com a empresa Alpha. Eles prometeram um 'retorno' legal pra gente por fora. Apaga essa mensagem depois."
]

## 2. Prompt Detector de Red Flags
Focamos em: Pressão indevida, contorno de controles, benefícios pessoais.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

class AnaliseRisco(BaseModel):
    risco: bool = Field(description="True se houver risco de fraude/compliance, False caso contrário")
    categoria: str = Field(description="Categoria: 'Pressão', 'Conluio', 'Bypass de Controle', 'Normal'")
    explicacao: str = Field(description="Explicação curta")

structured_llm = llm.with_structured_output(AnaliseRisco)

prompt = ChatPromptTemplate.from_template(
    """Analise o seguinte e-mail corporativo em busca de indicadores de fraude ou desvio de conduta.
    
    E-MAIL:
    {email}
    """
)

chain = prompt | structured_llm

## 3. Avaliação

Filtrando apenas os suspeitos.

In [ ]:
for email in emails:
    res = chain.invoke({"email": email})
    if res.risco:
        print(f"[SUSPEITO] {res.categoria}: {res.explicacao}")
        print(f"Trecho: {email}\n")

## Conclusão

Identificamos bypass de controle ("paga logo e depois vê nota") e conluio ("retorno por fora"). Ferramenta poderosa para e-discovery.